# Predicción de ventas para BlackFriday


## Problema

Un alto directivo de la empresa X está solicitando a nuestro equipo de ingenieros desarrollar un sistema que permita predecir información relevante para el próximo Blackfriday que se llevará a cabo en noviembre 23 del 2018, tomando como base los datos de ventas del blackfriday del año pasado que se llevó a cabo en noviembre 24 2017. 

El sistema debe ser capaz de responder las siguientes preguntas de forma gráfica y tabular (en tablas):

1. Unidades que se venderán por producto.
2. Total de ventas (valor de ventas) por producto en pesos. 
3. Top de compradores. 
4. ¿Nos comprarán más las personas solteras o las personas casadas? (en el dataset asumimos casadas como 1).
5. ¿Nos comprarán más los hombres o las mujeres? 
6. Clasifique por edades, que producto que se venderá más.

Para calcular los pesos se debe predecir el valor del dólar efectuando una regresión con los datos de dataset: TRM_Historico.xls 
* Se recomienda la librería [Bokeh](https://bokeh.pydata.org/en/latest/)

## Requerimientos

- El sistema debe usar ML para resolver las preguntas.
- Django 2.1 y Python 3.7 (librerías de ML)
- Postgress, en caso de requerir el uso de una BD

## Observaciones

#### TRM
La TRM es la Tasa Representativa del Mercado cambiario de Colombia y representa el valor que tiene un dólar estadounidense en pesos colombianos. 

#### Limitantes
- Detalle del crecimiento de la población:
La predicción se realiza con la información suministrada, y se asume que se tiene la misma población del año pasado y que se incrementarán las ventas. 
Esta predicción no toma en cuenta nuevos usuarios ni el crecimiento de la población de usuarios que accedio a la tienda en el último año.

Para solventar esta limitación el modelo se puede ajustar con un factor de crecimiento de población, es decir, un dato parametrizable ingresado por el usuario. Cabe aclarar, que este dato es un valor de crecimiento general y no discrimina por la demografia del usuario.

#### Versión de Django
Se actualiza la versión del _framework Django_ a la **2.1.2** debido a que en la versión **2.1** se reporto recientemente un fuerte problema de seguridad.



# Analisis exploratorio de los datos

In [2]:
import pandas as pd

df = pd.read_csv('datasets/BlackFriday.csv', sep=',')

In [3]:
df.sample(5)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
319047,1001181,P00119342,M,36-45,7,A,3,1,10,13.0,NaN,23579
30752,1004685,P00114842,M,36-45,1,B,2,1,5,NaN,NaN,3521
59641,1003224,P00046142,F,26-35,14,A,3,1,1,16.0,NaN,15182
366451,1002389,P00029242,M,26-35,16,A,0,1,8,NaN,NaN,7948
4401,1000721,P00207342,F,36-45,9,B,1,0,5,8.0,14.0,7165


In [5]:
columns = df.columns

In [11]:
h = df.head()
dic_pd = h.to_dict('split')

In [12]:
dic_pd

{'columns': ['User_ID',
  'Product_ID',
  'Gender',
  'Age',
  'Occupation',
  'City_Category',
  'Stay_In_Current_City_Years',
  'Marital_Status',
  'Product_Category_1',
  'Product_Category_2',
  'Product_Category_3',
  'Purchase'],
 'data': [[1000001,
   'P00069042',
   'F',
   '0-17',
   10,
   'A',
   '2',
   0,
   3,
   nan,
   nan,
   8370],
  [1000001, 'P00248942', 'F', '0-17', 10, 'A', '2', 0, 1, 6.0, 14.0, 15200],
  [1000001, 'P00087842', 'F', '0-17', 10, 'A', '2', 0, 12, nan, nan, 1422],
  [1000001, 'P00085442', 'F', '0-17', 10, 'A', '2', 0, 12, 14.0, nan, 1057],
  [1000002, 'P00285442', 'M', '55+', 16, 'C', '4+', 0, 8, nan, nan, 7969]],
 'index': [0, 1, 2, 3, 4]}

El conjunto de datos es de un solo dia y de un solo año no hay historico suficiente para predecir el numero de unidades que se venderan por producto. En este caso se puede hacer una suma del numero de productos vendidos agrupados por producto y tener este numero como referencia.

In [16]:
df.groupby(['Product_ID']).agg(['mean', 'count']).head()

User_ID       Occupation       Marital_Status        \
                    mean count       mean count           mean count   
Product_ID                                                             
P00000142   1.003035e+06  1130   7.972566  1130       0.375221  1130   
P00000242   1.003055e+06   371   7.582210   371       0.369272   371   
P00000342   1.002701e+06   238   7.869748   238       0.399160   238   
P00000442   1.003093e+06    92   6.847826    92       0.413043    92   
P00000542   1.003250e+06   146   7.239726   146       0.308219   146   

           Product_Category_1       Product_Category_2        \
                         mean count               mean count   
Product_ID                                                     
P00000142                   3  1130                4.0  1130   
P00000242                   2   371                4.0   371   
P00000342                   5   238               14.0   238   
P00000442                   5    92                8.0    92   
P00000542                   5   146                NaN     0   

           Product_Category_3            Purchase        
                         mean count          mean count  
Product_ID                                               
P00000142                 5.0  1130  11143.507080  1130  
P00000242                 9.0   371  10552.293801   371  
P00000342                 NaN     0   5299.928571   238  
P00000442                 NaN     0   4795.358696    92  
P00000542                 NaN     0   5419.308219   146

In [18]:
# total de ventas por producto en dolares
import numpy as np

In [22]:
sales_by_product = pd.DataFrame( { 'sales': df[['Product_ID','Purchase']].groupby(['Product_ID'])['Purchase'].transform(np.sum) }) 

sales_by_product['pesos'] = sales_by_product['sales'].apply(lambda x: x*2999)


In [23]:
sales_by_product.head()

,sales,pesos
0,2621891,7863051109
1,9287185,27852267815
2,122792,368253208
3,485991,1457487009
4,1543739,4629673261


In [24]:
purchase_by_user = pd.DataFrame( { 'amount': df[['User_ID','Purchase']].groupby(['User_ID'])['Purchase'].transform(np.sum) }) 

purchase_by_user['pesos'] = purchase_by_user['amount'].apply(lambda x: x*2999)
purchase_by_user.sort_values('amount', ascending=False)
purchase_by_user.head(10)

,amount,pesos
0,333481,1000109519
1,333481,1000109519
2,333481,1000109519
3,333481,1000109519
4,810353,2430248647
5,341635,1024563365
6,205987,617755013
7,205987,617755013
8,205987,617755013
9,821001,2462181999


In [25]:
df.head(3)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 12 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null object
Gender                        537577 non-null object
Age                           537577 non-null object
Occupation                    537577 non-null int64
City_Category                 537577 non-null object
Stay_In_Current_City_Years    537577 non-null object
Marital_Status                537577 non-null int64
Product_Category_1            537577 non-null int64
Product_Category_2            370591 non-null float64
Product_Category_3            164278 non-null float64
Purchase                      537577 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 49.2+ MB


### Previsualización del dataframe

Comenzamos inspeccionando los tipos de datos del dataset e identificamos cuales de ellos tienen valores nulos.

- Tenemos un total de 537.577 registros
- Las variables *Product_Category_2* y *Product_Category_3* son las únicas que tienen valores nulos, pero para nuestro analisis estas variables no son necesarias
- La variable **Age** es una variable que se debe normalizar


In [ ]:
# previsualizando el dataframe
df.head(10)

In [ ]:
# previsualizando el dataframe
df.tail(10)

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# listado de columnas que nos interesan para crear un nuevo dataframe
list_columns = ['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 
                'Stay_In_Current_City_Years', 'Marital_Status', 'Purchase']

In [ ]:
!pip install xlrd

In [27]:
dolars = pd.read_excel('datasets/TRM_Historico.xls')

In [28]:
dolars.head()

,Fecha,TRM
0,1991-11-27 00:00:00,693.32
1,1991-11-28 00:00:00,693.99
2,1991-11-29 00:00:00,694.70
3,1991-11-30 00:00:00,694.70
4,1991-12-01 00:00:00,643.42


In [29]:
dolars.loc[:, 'Fecha'].head()


0    1991-11-27 00:00:00
1    1991-11-28 00:00:00
2    1991-11-29 00:00:00
3    1991-11-30 00:00:00
4    1991-12-01 00:00:00
Name: Fecha, dtype: object

In [30]:
# vamos a normalizar los datos

gender_dict = {'F':0, 'M':1}
age_dict = {'0-17':0, '18-25':1, '26-35':2, '36-45':3, '46-50':4, '51-55':5, '55+':6}
city_dict = {'A':0, 'B':1, 'C':2}
stay_dict = {'0':0, '1':1, '2':2, '3':3, '4+':4}

In [31]:
# eliminando columnas con datos vacios
df = df.drop(["Product_Category_2"], axis=1)
df = df.drop(["Product_Category_3"], axis=1)

In [32]:
df["Gender"] = df["Gender"].apply(lambda x: gender_dict[x])

df["Age"] = df["Age"].apply(lambda x: age_dict[x])

df["City_Category"] = df["City_Category"].apply(lambda x: city_dict[x])

df["Stay_In_Current_City_Years"] = df["Stay_In_Current_City_Years"].apply(lambda x: stay_dict[x])

In [33]:
df.sample(8)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase
173827,1002901,P00101842,1,2,17,1,2,0,6,20520
511583,1000840,P00210942,0,2,3,2,1,1,5,8630
337978,1004013,P00261142,1,2,1,2,4,1,1,7734
164850,1001451,P00100842,1,3,20,1,0,0,5,8627
426402,1005648,P00110842,1,2,17,2,1,0,1,15855
298016,1003888,P00075542,1,4,17,2,0,1,8,9991
362558,1001794,P00113642,1,2,11,1,1,0,8,2314
343089,1004796,P00119342,1,3,7,0,4,1,10,19110
